<h2>Capsule Network on the classic IMDB Sentiment</h2>
<p>The goal of this notebook is to explore the Capsule Network on the classic IMDB Sentiment Analysis dataset. I would like to investigate how well the CapsNet does on NLP task such as sentiment analysis and compare it to the state of the art.</p>

In [175]:
### Import Libraries
import pandas as pd
import numpy as np
import gensim

import torch
import torch.nn as nn

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors

In [157]:
### Settings to tokenize sentences and convert labels to torch floats
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

### Get test/train split for torchtext
train, test = datasets.IMDB.splits(TEXT, LABEL)

In [158]:
print('Number of training examples: {}'.format(len(train)))
print('Number of training examples: {}'.format(len(test)))

Number of training examples: 25000
Number of training examples: 25000


In [159]:
### Example Review and Label
print(vars(train_data.examples[1]))

{'text': ['This', 'movie', 'had', 'me', 'smiling', 'from', 'beginning', 'to', 'end', ',', 'partly', 'at', 'the', 'humor', ',', 'partly', 'at', 'Meg', 'Ryan', '(', 'this', 'is', 'the', 'perfect', 'character', 'for', 'her', ')', ',', 'and', 'always', 'because', 'it', "'s", 'just', 'one', 'of', 'the', 'best', 'feel', '-', 'good', 'movies', 'I', "'ve", 'seen', '.', 'Hopefully', 'the', 'DVD', 'will', 'be', 'out', 'soon', '.'], 'label': 'pos'}


In [160]:
model = gensim.models.Word2Vec('./models/GoogleNews-vectors-negative300.bin')
model.wv.save_word2vec_format('./models/GooglePretrain.model')

In [161]:
### Load Google Word2Vec for word embeddings
vectors = Vectors('./models/GooglePretrain.model')

In [166]:
# build the vocabulary
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [177]:
### Create embedding layer
TEXT.vocab.set_vectors(vectors.stoi, vectors.vectors, vectors.dim)
embedding = nn.Embedding.from_pretrained(torch.FloatTensor(TEXT.vocab.vectors))

In [168]:
### BucketIterator which is a special type of iterator that will return a batch of examples where each example is of a similar length, minimizing the amount of padding per example
BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, test_iter = data.BucketIterator.splits(
    (train, test), batch_size=BATCH_SIZE, device=device)